In [1]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize
import os

In [2]:
# CONFIGURATION
BASE_DIR = r"C:\Users\ayber\OneDrive\Masaüstü\ML"
FILE_PATH = os.path.join(BASE_DIR, "Data/external/datasetIRB2400.csv")
OUTPUT_PATH = os.path.join(BASE_DIR, "Data/processed/irb2400_ready_v2.csv") # New version

In [3]:
# ROBOT CLASS (Optimized) for FK
class IRB2400_Fast:
    def __init__(self, offset_q2, offset_q3, a3, d6, tool_length):
        self.d1 = 0.615
        self.a1 = 0.100
        self.a2 = 0.705
        self.offset_q2 = offset_q2
        self.offset_q3 = offset_q3
        self.a3 = a3
        self.d4 = 0.755
        self.d6 = d6 + tool_length 

    def single_fk(self, q):
        def dh(theta, d, a, alpha):
            t = np.radians(theta)
            alp = np.radians(alpha)
            c, s = np.cos(t), np.sin(t)
            ca, sa = np.cos(alp), np.sin(alp)
            return np.array([
                [c, -s*ca, s*sa, a*c],
                [s, c*ca, -c*sa, a*s],
                [0, sa,    ca,   d],
                [0, 0,     0,    1]
            ])
        
        T01 = dh(q[0],                self.d1, self.a1, -90)
        T12 = dh(q[1] + self.offset_q2, 0,       self.a2, 0)
        T23 = dh(q[2] + self.offset_q3, 0,       self.a3, -90)
        T34 = dh(q[3],                self.d4, 0,       90)
        T45 = dh(q[4],                0,       0,       -90)
        T56 = dh(q[5],                self.d6, 0,       0)
        
        T = T01 @ T12 @ T23 @ T34 @ T45 @ T56
        return T[:3, 3]

def process_and_finalize():
    print(f"Loading {FILE_PATH}...")
    df = pd.read_csv(FILE_PATH)
    
    # 1. PREPARE DATA
    joint_cols_out = ['q1_out', 'q2_out', 'q3_out', 'q4_out', 'q5_out', 'q6_out']
    q_out_rad = df[joint_cols_out].values
    q_out_deg = np.degrees(q_out_rad)
    
    target_pos_mm = df[['x', 'y', 'z']].values
    target_pos_m = target_pos_mm / 1000.0
    
    # 2. OPTIMIZATION (Reuse previous best values to save time)
    # We found: a3=0.0315, d6=0.1547
    print("Using Optimized Geometry (a3=0.0315, d6=0.1547)...")
    
    # 3. CALCULATE DELTAS
    joint_cols_in = ['q1_in', 'q2_in', 'q3_in', 'q4_in', 'q5_in', 'q6_in']
    q_in_rad = df[joint_cols_in].values
    q_in_deg = np.degrees(q_in_rad)
    
    delta_q = q_out_deg - q_in_deg
    delta_q = (delta_q + 180) % 360 - 180 # Wrap around
    
    # 4. SAVE NEW DATASET WITH SIN/COS FEATURES
    print("Generating Sin/Cos Features...")
    df_new = pd.DataFrame()
    
    # Cartesian Position
    df_new['target_x'] = target_pos_m[:,0]
    df_new['target_y'] = target_pos_m[:,1]
    df_new['target_z'] = target_pos_m[:,2]
    
    # Orientation: Sin/Cos Encoding (Better for NN)
    # df['yaw'] is in radians
    df_new['sin_yaw'] = np.sin(df['yaw'])
    df_new['cos_yaw'] = np.cos(df['yaw'])
    df_new['sin_pitch'] = np.sin(df['pitch'])
    df_new['cos_pitch'] = np.cos(df['pitch'])
    df_new['sin_roll'] = np.sin(df['roll'])
    df_new['cos_roll'] = np.cos(df['roll'])
    
    # Current Joints
    for i in range(6):
        df_new[f'q{i+1}_in'] = q_in_deg[:,i]
    
    # Targets (Delta)
    for i in range(6):
        df_new[f'delta_q{i+1}'] = delta_q[:,i]
        
    # Reference (True Targets)
    for i in range(6):
        df_new[f'q{i+1}_out'] = q_out_deg[:,i]

    # Save
    os.makedirs(os.path.dirname(OUTPUT_PATH), exist_ok=True)
    df_new.to_csv(OUTPUT_PATH, index=False)
    print(f"Processed dataset saved to: {OUTPUT_PATH}")
    print("New Columns include: sin_yaw, cos_yaw, etc.")

if __name__ == "__main__":
    process_and_finalize()

Loading C:\Users\ayber\OneDrive\Masaüstü\ML\Data/external/datasetIRB2400.csv...
Using Optimized Geometry (a3=0.0315, d6=0.1547)...
Generating Sin/Cos Features...
Processed dataset saved to: C:\Users\ayber\OneDrive\Masaüstü\ML\Data/processed/irb2400_ready_v2.csv
New Columns include: sin_yaw, cos_yaw, etc.
